Code From: https://towardsdatascience.com/implementing-alexnet-cnn-architecture-using-tensorflow-2-0-and-keras-2113e090ad98

In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import time
import pickle
from PIL import Image
import numpy as np
import glob
import pandas as pd

In [2]:
train_images = pickle.load(open("/Users/chris/Documents/GitHub/Stat440-module3/ChrisL/train_images.pickle","rb"))
train_labels = pickle.load(open("/Users/chris/Documents/GitHub/Stat440-module3/ChrisL/train_labels.pickle","rb"))
validation_images = pickle.load(open("/Users/chris/Documents/GitHub/Stat440-module3/ChrisL/validation_images.pickle","rb"))
validation_labels = pickle.load(open("/Users/chris/Documents/GitHub/Stat440-module3/ChrisL/validation_labels.pickle","rb"))
test_images = pickle.load(open("/Users/chris/Documents/GitHub/Stat440-module3/ChrisL/test_images.pickle","rb"))

In [3]:
train_images.shape
validation_images.shape

(100, 384, 512, 3)

In [4]:
train_labels = train_labels - 1
validation_labels = validation_labels - 1
test_labels = np.zeros(len(test_images),dtype = "uint8").reshape(-1,1)

In [5]:
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
validation_ds = tf.data.Dataset.from_tensor_slices((validation_images, validation_labels))

In [6]:
def process_images(image, label):
    # Normalize images to have a mean of 0 and standard deviation of 1
    image = tf.image.per_image_standardization(image)
    # Resize images from 32x32 to 277x277
    image = tf.image.resize(image, (227,227))
    return image, label

In [7]:
train_ds_size = tf.data.experimental.cardinality(train_ds).numpy()
test_ds_size = tf.data.experimental.cardinality(test_ds).numpy()
validation_ds_size = tf.data.experimental.cardinality(validation_ds).numpy()
print("Training data size:", train_ds_size)
print("Test data size:", test_ds_size)
print("Validation data size:", validation_ds_size)

Training data size: 1163
Test data size: 1264
Validation data size: 100


In [8]:
train_ds = (train_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=32))
test_ds = (test_ds
                  .map(process_images)
                  .batch(batch_size=32))
validation_ds = (validation_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=32))

In [9]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(227,227,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(6, activation='softmax')
])

In [10]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.optimizers.SGD(lr=0.005), metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 55, 55, 96)        34944     
_________________________________________________________________
batch_normalization (BatchNo (None, 55, 55, 96)        384       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 27, 27, 256)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 384)       8

In [11]:
root_logdir = os.path.join(os.curdir, "logs\\fit\\")
def get_run_logdir():
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)
run_logdir = get_run_logdir()
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = earlystop

In [12]:
model.fit(train_ds,
          epochs=50,
          validation_data=validation_ds,
          validation_freq=1,
          callbacks=[tensorboard_cb])

Epoch 1/50
37/37 [==============================] - 33s 800ms/step - loss: 5.7798 - accuracy: 0.2671 - val_loss: 3.4041 - val_accuracy: 0.2400
Epoch 2/50
37/37 [==============================] - 31s 781ms/step - loss: 1.9587 - accuracy: 0.4005 - val_loss: 1.9329 - val_accuracy: 0.4500
Epoch 3/50
37/37 [==============================] - 31s 786ms/step - loss: 1.3554 - accuracy: 0.5240 - val_loss: 1.4276 - val_accuracy: 0.4900
Epoch 4/50
37/37 [==============================] - 32s 823ms/step - loss: 1.2758 - accuracy: 0.5863 - val_loss: 1.7182 - val_accuracy: 0.4300
Epoch 5/50
37/37 [==============================] - 33s 821ms/step - loss: 1.0696 - accuracy: 0.5938 - val_loss: 1.0713 - val_accuracy: 0.5400
Epoch 6/50
37/37 [==============================] - 34s 843ms/step - loss: 0.9356 - accuracy: 0.6480 - val_loss: 1.1417 - val_accuracy: 0.5800
Epoch 7/50
37/37 [==============================] - 33s 818ms/step - loss: 0.7291 - accuracy: 0.7059 - val_loss: 0.9353 - val_accuracy: 0.6800

In [13]:
prediction = model.predict(test_ds)

In [14]:
yPred = np.argmax(prediction, axis=-1)

In [15]:
model.evaluate(validation_ds)

4/4 [==============================] - 1s 204ms/step - loss: 0.9598 - accuracy: 0.7100


[0.9597627520561218, 0.7099999785423279]

In [16]:
pred_df = pd.read_csv("/Users/chris/Documents/GitHub/Stat440-module3/ChrisL/example.csv")

In [17]:
pred_df['Prediction'] = yPred
pred_df['Prediction'] = pred_df['Prediction'] + 1

In [18]:
pred_df.to_csv("/Users/chris/Documents/GitHub/Stat440-module3/ChrisL/prediction.csv",index=False)